Многослойный персептрон
- Реализовать архитектуру многослойного персептрона. 
- Реализовать алгоритм обратного распространения ошибки для обучения сети;
- Обучить на очищенном датасете  из 1-ой лабораторной. 
- Реализовать перцептрон - регрессор; 
- Обработать датасет с классификацией грибов по алгоритму из 1-й лабораторной работы.
- Обучить перцептрон на датасете с классификацией грибов. 
- Реализовать классификатор.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import sklearn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.neural_network import MLPClassifier, MLPRegressor

# Перцептрон - Регрессор (ноутбуки)

In [2]:
ldf = pd.read_csv("./Laptop_price.csv")

In [3]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             1000 non-null   object 
 1   Processor_Speed   1000 non-null   float64
 2   RAM_Size          1000 non-null   int64  
 3   Storage_Capacity  1000 non-null   int64  
 4   Screen_Size       1000 non-null   float64
 5   Weight            1000 non-null   float64
 6   Price             1000 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 54.8+ KB


In [4]:
ldf['Brand'].unique()

array(['Asus', 'Acer', 'Lenovo', 'HP', 'Dell'], dtype=object)

In [5]:
ldf = pd.get_dummies(ldf,columns=['Brand'])

In [6]:
ldf.head()

,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price,Brand_Acer,Brand_Asus,Brand_Dell,Brand_HP,Brand_Lenovo
0,3.830296,16,512,11.185147,2.641094,17395.093065,False,True,False,False,False
1,2.912833,4,1000,11.311372,3.260012,31607.605919,True,False,False,False,False
2,3.241627,4,256,11.853023,2.029061,9291.023542,False,False,False,False,True
3,3.806248,16,512,12.280360,4.573865,17436.728334,True,False,False,False,False
4,3.268097,32,1000,14.990877,4.193472,32917.990718,True,False,False,False,False


In [7]:
X = ldf.drop(columns=['Price']).values
y = ldf['Price'].values.reshape(-1, 1)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)


In [10]:
#X_train = torch.tensor(X_train, dtype=torch.float32)
#X_test = torch.tensor(X_test, dtype=torch.float32)
#y_train = torch.tensor(y_train, dtype=torch.float32)
#y_test = torch.tensor(y_test, dtype=torch.float32)

## Своя модель

### Код

In [11]:
# Линейный слой
class Linear:
  def __init__(self, input_size=None, output_size=None):
    self.X = None
    self.input_size = input_size
    self.output_size = output_size
    self.W = np.random.randn(input_size, output_size)
    self.bias = np.zeros((1, output_size))

  def __call__(self, X):
    self.X = X
    result = np.dot(X, self.W) + self.bias
    return result

  def prop(self, grad_prev, learning_rate):
    grad_X = np.dot(grad_prev, self.W.T)
    grad_W = np.dot(self.X.T, grad_prev)
    grad_bias = np.sum(grad_prev, axis=0)

    self.W -= learning_rate * grad_W
    self.bias -= learning_rate * grad_bias
    return grad_X



In [12]:
# Функция активации
class Sigmoid:
  def __init__(self):
    self.y = None

  def __call__(self, x):
    self.y = self.sigmoid(x)
    return self.y


  def sigmoid(self,x):
      return 1. / (1. + np.exp(-x))
    
  def sigmoid_deriv(self, x):
      return x * (1 - x)
    
  def prop(self, grad_prev, learning_rate):
    return grad_prev * self.sigmoid_deriv(self.y)

In [13]:
# Перцептрон 
class MLP:
  def __init__(self, layers, max_iter=100, batch_size=100, learning_rate=0.001):
    self.layers = layers
    self.max_iter = max_iter
    self.batch_size = batch_size
    self.learning_rate = learning_rate

  @staticmethod
  def score(y_true, y_pred):
    if np.sum((y_true - np.mean(y_true)) ** 2) == 0:
      return 1
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))


  def predict(self, input_data):
    prev_data = input_data
    for layer in self.layers:
      prev_data = layer(prev_data)
    return prev_data

  def _prop(self, grad_prev, learning_rate):
    for layer in reversed(self.layers):
      grad_prev = layer.prop(grad_prev, learning_rate)

  def fit(self, X, y):
    for iter in range(self.max_iter):
      permutation = np.random.permutation(len(X))
      X_shuffled = X[permutation]
      y_shuffled = y[permutation]

      total_loss = 0.
      for batch_first in range(0, len(X), self.batch_size):
        batch_last = min(batch_first + self.batch_size, len(X))

        X_batch = X_shuffled[batch_first:batch_last]
        y_batch = y_shuffled[batch_first:batch_last]

        y_pred = self.predict(X_batch)
        total_loss += self._loss(y_batch, y_pred)
        grad_prev = self._grad_loss(y_batch, y_pred)
        self._prop(grad_prev, self.learning_rate)

      print(f'iteration: {iter}, loss: {total_loss}')

  @staticmethod
  def _loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

  @staticmethod
  def _grad_loss(y_true, y_pred):
    return 2 * (y_pred - y_true) / len(y_true)

In [14]:
class Normalizer:
  def __init__(self):
    self.mean = None
    self.std = None

  def fit_transform(self, data):
    self.mean = np.mean(data, axis=0)
    self.std = np.std(data, axis=0)
    return (data - self.mean) / self.std


### Обучение

In [15]:
df = pd.read_csv('Laptop_price.csv')
print(df.head())

X = df[['Storage_Capacity', 'RAM_Size', 'Processor_Speed', 'Screen_Size', 'Weight']].values
y = df['Price'].values.reshape(-1, 1)

normalizerX = Normalizer()
normalizerY = Normalizer()

X = normalizerX.fit_transform(X)
y = normalizerY.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    Brand  Processor_Speed  RAM_Size  Storage_Capacity  Screen_Size    Weight  \
0    Asus         3.830296        16               512    11.185147  2.641094   
1    Acer         2.912833         4              1000    11.311372  3.260012   
2  Lenovo         3.241627         4               256    11.853023  2.029061   
3    Acer         3.806248        16               512    12.280360  4.573865   
4    Acer         3.268097        32              1000    14.990877  4.193472   

          Price  
0  17395.093065  
1  31607.605919  
2   9291.023542  
3  17436.728334  
4  32917.990718  


In [16]:

model = MLP([
    Linear(X_train.shape[1], 10),
    Sigmoid(),
    Linear(10, 30),
    Sigmoid(),
    Linear(30, 1)
], learning_rate=0.001, max_iter=1000)
model.fit(X_train, y_train)

iteration: 0, loss: 18.53261801973367
iteration: 1, loss: 18.14225683722818
iteration: 2, loss: 17.77267800837324
iteration: 3, loss: 17.414374384555018
iteration: 4, loss: 17.06467939219755
iteration: 5, loss: 16.724763616430057
iteration: 6, loss: 16.39939710333265
iteration: 7, loss: 16.08004346274014
iteration: 8, loss: 15.767853486999007
iteration: 9, loss: 15.467843309711494
iteration: 10, loss: 15.174854985461053
iteration: 11, loss: 14.885682631227775
iteration: 12, loss: 14.608469859720495
iteration: 13, loss: 14.339119339916657
iteration: 14, loss: 14.075144730968647
iteration: 15, loss: 13.816213439751854
iteration: 16, loss: 13.566263672124958
iteration: 17, loss: 13.32374507715716
iteration: 18, loss: 13.083060774684165
iteration: 19, loss: 12.852175321189492
iteration: 20, loss: 12.625914037829286
iteration: 21, loss: 12.405242717242968
iteration: 22, loss: 12.18912621784666
iteration: 23, loss: 11.978822228120436
iteration: 24, loss: 11.77489883852256
iteration: 25, loss

In [17]:
y_exp = y_test.T[0]
y_pred = model.predict(X_test).T[0]
model.score(y_exp,y_pred)

0.9533029253943877

# Перцептрон - Классификатор (грибы)

In [18]:
gdf = pd.read_csv("./mushroom/expanded.csv")

In [19]:
gdf.replace("?", pd.NA, inplace=True)

In [20]:
nans = gdf.isna().sum()
print(nans[nans != 0])

stalk-root    2480
dtype: int64


In [21]:
gdf['stalk-root'].fillna(gdf['stalk-root'].mode()[0], inplace=True)

/tmp/ipykernel_63438/4140029336.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf['stalk-root'].fillna(gdf['stalk-root'].mode()[0], inplace=True)


In [22]:
nans = gdf.isna().sum()
print(nans[nans != 0])

Series([], dtype: int64)


In [23]:
u =gdf.apply(lambda x: x.unique())
u

class                                                     [EDIBLE, POISONOUS]
cap-shape                      [CONVEX, FLAT, BELL, SUNKEN, KNOBBED, CONICAL]
cap-surface                                 [SMOOTH, FIBROUS, SCALY, GROOVES]
cap-color                   [WHITE, YELLOW, BROWN, GRAY, RED, PINK, PURPLE...
brusies                                                         [BRUISES, NO]
odor                        [ALMOND, ANISE, NONE, PUNGENT, CREOSOTE, FOUL,...
gill-attachment                                              [FREE, ATTACHED]
gill-spacing                                                 [CROWDED, CLOSE]
gill-size                                                     [NARROW, BROAD]
gill-color                  [WHITE, PINK, BROWN, GRAY, BLACK, CHOCOLATE, P...
stalk-shape                                             [TAPERING, ENLARGING]
stalk-root                                     [BULBOUS, CLUB, ROOTED, EQUAL]
stalk-surface-above-ring                      [SMOOTH, FIBROUS, 

In [24]:
X = gdf.drop(columns=['class'])
y = gdf['class']

In [25]:
cat_features = [x for x in X.columns]
ohe_features = [x for x in X.columns]

In [26]:
for col in cat_features:
    if (len(X[col].unique()) == 2):
        X[col] = (X[col] == X[col][0]).astype(int)
        ohe_features.remove(col)
        

In [27]:
ring_mapping = {'NONE':0, 'ONE': 1, 'TWO': 2}
X['ring-number'] = X['ring-number'].map(ring_mapping)
ohe_features.remove('ring-number')


In [28]:
pd.set_option('display.max_columns', None)
X.sample(n=20)

,cap-shape,cap-surface,cap-color,brusies,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
6733,FLAT,SMOOTH,RED,0,FISHY,1,0,1,BUFF,1,BULBOUS,SMOOTH,SMOOTH,WHITE,PINK,PARTIAL,WHITE,1,EVANESCENT,WHITE,SEVERAL,WOODS
4185,CONVEX,FIBROUS,GRAY,0,FOUL,1,0,0,PINK,0,BULBOUS,SILKY,SILKY,PINK,BROWN,PARTIAL,WHITE,1,LARGE,CHOCOLATE,SEVERAL,WOODS
3022,CONVEX,SCALY,BROWN,1,NONE,1,0,0,BROWN,1,BULBOUS,SMOOTH,SMOOTH,PINK,PINK,PARTIAL,WHITE,1,PENDANT,BROWN,SOLITARY,WOODS
8256,BELL,SMOOTH,BROWN,0,NONE,0,0,0,BROWN,0,BULBOUS,SMOOTH,SMOOTH,ORANGE,ORANGE,PARTIAL,ORANGE,1,PENDANT,YELLOW,SEVERAL,LEAVES
8209,FLAT,SMOOTH,GRAY,1,NONE,1,0,0,WHITE,0,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,2,PENDANT,WHITE,SEVERAL,PATHS
6643,CONVEX,SCALY,BROWN,0,SPICY,1,0,1,BUFF,1,BULBOUS,SMOOTH,SMOOTH,PINK,PINK,PARTIAL,WHITE,1,EVANESCENT,WHITE,SEVERAL,WOODS
7584,KNOBBED,SCALY,RED,0,FOUL,1,0,1,BUFF,1,BULBOUS,SILKY,SILKY,WHITE,PINK,PARTIAL,WHITE,1,EVANESCENT,WHITE,SEVERAL,PATHS
6330,CONVEX,SMOOTH,BROWN,0,FISHY,1,0,1,BUFF,1,BULBOUS,SILKY,SMOOTH,PINK,PINK,PARTIAL,WHITE,1,EVANESCENT,WHITE,SEVERAL,PATHS
3408,FLAT,FIBROUS,BROWN,1,NONE,1,0,0,PURPLE,1,BULBOUS,SMOOTH,SMOOTH,WHITE,GRAY,PARTIAL,WHITE,1,PENDANT,BLACK,SOLITARY,WOODS
7855,FLAT,SCALY,BROWN,0,MUSTY,0,0,0,YELLOW,0,CLUB,SILKY,SCALY,CINNAMON,CINNAMON,PARTIAL,WHITE,0,NONE,WHITE,CLUSTERED,WOODS


In [29]:
X = pd.get_dummies(X, columns=ohe_features)

In [30]:
print(X.shape)
X.sample(n=5)


(8416, 109)


,brusies,gill-attachment,gill-spacing,gill-size,stalk-shape,ring-number,cap-shape_BELL,cap-shape_CONICAL,cap-shape_CONVEX,cap-shape_FLAT,cap-shape_KNOBBED,cap-shape_SUNKEN,cap-surface_FIBROUS,cap-surface_GROOVES,cap-surface_SCALY,cap-surface_SMOOTH,cap-color_BROWN,cap-color_BUFF,cap-color_CINNAMON,cap-color_GRAY,cap-color_GREEN,cap-color_PINK,cap-color_PURPLE,cap-color_RED,cap-color_WHITE,cap-color_YELLOW,odor_ALMOND,odor_ANISE,odor_CREOSOTE,odor_FISHY,odor_FOUL,odor_MUSTY,odor_NONE,odor_PUNGENT,odor_SPICY,gill-color_BLACK,gill-color_BROWN,gill-color_BUFF,gill-color_CHOCOLATE,gill-color_GRAY,gill-color_GREEN,gill-color_ORANGE,gill-color_PINK,gill-color_PURPLE,gill-color_RED,gill-color_WHITE,gill-color_YELLOW,stalk-root_BULBOUS,stalk-root_CLUB,stalk-root_EQUAL,stalk-root_ROOTED,stalk-surface-above-ring_FIBROUS,stalk-surface-above-ring_SCALY,stalk-surface-above-ring_SILKY,stalk-surface-above-ring_SMOOTH,stalk-surface-below-ring_FIBROUS,stalk-surface-below-ring_SCALY,stalk-surface-below-ring_SILKY,stalk-surface-below-ring_SMOOTH,stalk-color-above-ring_BROWN,stalk-color-above-ring_BUFF,stalk-color-above-ring_CINNAMON,stalk-color-above-ring_GRAY,stalk-color-above-ring_ORANGE,stalk-color-above-ring_PINK,stalk-color-above-ring_RED,stalk-color-above-ring_WHITE,stalk-color-above-ring_YELLOW,stalk-color-below-ring_BROWN,stalk-color-below-ring_BUFF,stalk-color-below-ring_CINNAMON,stalk-color-below-ring_GRAY,stalk-color-below-ring_ORANGE,stalk-color-below-ring_PINK,stalk-color-below-ring_RED,stalk-color-below-ring_WHITE,stalk-color-below-ring_YELLOW,veil-type_PARTIAL,veil-color_BROWN,veil-color_ORANGE,veil-color_WHITE,veil-color_YELLOW,ring-type_EVANESCENT,ring-type_FLARING,ring-type_LARGE,ring-type_NONE,ring-type_PENDANT,spore-print-color_BLACK,spore-print-color_BROWN,spore-print-color_BUFF,spore-print-color_CHOCOLATE,spore-print-color_GREEN,spore-print-color_ORANGE,spore-print-color_PURPLE,spore-print-color_WHITE,spore-print-color_YELLOW,population_ABUNDANT,population_CLUSTERED,population_NUMEROUS,population_SCATTERED,population_SEVERAL,population_SOLITARY,habitat_GRASSES,habitat_LEAVES,habitat_MEADOWS,habitat_PATHS,habitat_URBAN,habitat_WASTE,habitat_WOODS
7981,0,1,1,0,0,2,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False
4720,0,1,0,0,0,1,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False
6381,0,1,0,1,1,1,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,Fals

In [31]:
class_mapping = {'EDIBLE': 1, 'POISONOUS': 0}
y = y.map(class_mapping)



In [32]:
y.sample(n=5)

3766    1
96      1
8200    1
3443    1
5779    0
Name: class, dtype: int64

In [34]:
X = X.values
y = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
#X_train = torch.tensor(X_train, dtype=torch.float32)
#X_test = torch.tensor(X_test, dtype=torch.float32)
#y_train = torch.tensor(y_train, dtype=torch.float32)
#y_test = torch.tensor(y_test, dtype=torch.float32)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

## Своя модель

In [35]:
gdf = pd.read_csv("./mushroom/expanded.csv")
gdf.replace("?", pd.NA, inplace=True)
gdf['stalk-root'].fillna(gdf['stalk-root'].mode()[0], inplace=True)
u =gdf.apply(lambda x: x.unique())
X = gdf.drop(columns=['class'])
y = gdf['class']
cat_features = [x for x in X.columns]
ohe_features = [x for x in X.columns]
for col in cat_features:
    if (len(X[col].unique()) == 2):
        X[col] = (X[col] == X[col][0]).astype(int)
        ohe_features.remove(col)
ring_mapping = {'NONE':0, 'ONE': 1, 'TWO': 2}
X['ring-number'] = X['ring-number'].map(ring_mapping)
ohe_features.remove('ring-number')
X = pd.get_dummies(X, columns=ohe_features)
class_mapping = {'EDIBLE': 1, 'POISONOUS': 0}
y = y.map(class_mapping)





/tmp/ipykernel_63438/2261711917.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf['stalk-root'].fillna(gdf['stalk-root'].mode()[0], inplace=True)


In [37]:
mushrooms = fetch_ucirepo(id=73)


In [38]:
df = mushrooms.data.features
target = mushrooms.data.targets['poisonous']

df['stalk-root'].fillna('m', inplace=True)
X = df

encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X).toarray()
y_encoded = target.apply(lambda x: 0 if x == 'p' else 1).values.reshape(-1, 1)



/tmp/ipykernel_63438/2629846134.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['stalk-root'].fillna('m', inplace=True)
/tmp/ipykernel_63438/2629846134.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stalk-root'].fillna('m', inplace=True)


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=65)



In [40]:
model = MLP([
    Linear(X_train.shape[1], 100),
    Sigmoid(),
    Linear(100, 1),
    Sigmoid()
], learning_rate=0.05, max_iter=100)
model.fit(X_train, y_train)

iteration: 0, loss: 26.285152511498136
iteration: 1, loss: 20.382204679945374
iteration: 2, loss: 16.235462386375627
iteration: 3, loss: 12.694856336842244
iteration: 4, loss: 10.345479299226346
iteration: 5, loss: 8.929055780540645
iteration: 6, loss: 8.038134317980655
iteration: 7, loss: 7.444227284424475
iteration: 8, loss: 7.015253125435622
iteration: 9, loss: 6.688744967189931
iteration: 10, loss: 6.423168543490324
iteration: 11, loss: 6.19992965226865
iteration: 12, loss: 6.001164616230405
iteration: 13, loss: 5.820253132703173
iteration: 14, loss: 5.65114408302831
iteration: 15, loss: 5.487445354208369
iteration: 16, loss: 5.328619796117897
iteration: 17, loss: 5.170643607216995
iteration: 18, loss: 5.016811607141028
iteration: 19, loss: 4.86411898400066
iteration: 20, loss: 4.711674589538198
iteration: 21, loss: 4.567538287650464
iteration: 22, loss: 4.424032045529015
iteration: 23, loss: 4.285303584888445
iteration: 24, loss: 4.150412966364809
iteration: 25, loss: 4.0204756335

In [41]:
y_exp = y_test.T[0]
y_pred = model.predict(X_test).T[0]
model.score(y_exp, y_pred)

0.9415309112703624